In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
import os
from math import cos, sin, pi, sqrt
from math import radians
%matplotlib inline

In [7]:
def split_in_blocks(im_raw, n_blocks=(4,4), overlap=(448,448)):
    h, w = im_raw.shape[:2]
    w_block = (w + (n_blocks[1] - 1) * overlap[1]) // n_blocks[1]
    h_block = (h + (n_blocks[0] - 1) * overlap[0]) // n_blocks[0]
    
    if (w + (n_blocks[1] - 1) * overlap[1]) % n_blocks[1] != 0 or \
        (h + (n_blocks[0] - 1) * overlap[0]) % n_blocks[0] != 0:
            print("Warning: check the number of blocks and overlap")
            
    for by in range(n_blocks[0]):
        y_start = by * (h_block - overlap[0])
        y_end = y_start + h_block + 1
        for bx in range(n_blocks[1]):
            x_start = bx * (w_block - overlap[1])
            x_end = x_start + w_block + 1
            yield im_raw[y_start:y_end, x_start:x_end,...]

In [12]:
def check_img(root_dir):
    raw_dir = os.path.join(root_dir, "Train")
    mask_dir = os.path.join(root_dir, "TrainMask")
    
    for filename in os.listdir(raw_dir):
        if not filename.endswith(".jpg"):
            continue

        train_id, _ = os.path.splitext(filename)
        # load the raw image, mask and dots
        im_raw = cv2.imread(os.path.join(raw_dir, filename))
        im_mask = cv2.imread(os.path.join(mask_dir, train_id + ".png"), cv2.IMREAD_GRAYSCALE)

        if im_raw.shape[:2] != im_mask.shape[:2]:
            print(train_id)

In [13]:
check_img(root_dir)

530
638


In [14]:
def generate_training_blocks(root_dir):
    raw_dir = os.path.join(root_dir, "Train")
    mask_dir = os.path.join(root_dir, "TrainMask")
    block_dir = os.path.join(root_dir, "TrainBlock")
    
    os.makedirs(block_dir, exist_ok=True)
    
    for filename in os.listdir(raw_dir):
        if not filename.endswith(".jpg"):
            continue
        
        train_id, _ = os.path.splitext(filename)
        # load the raw image, mask and dots
        im_raw = cv2.imread(os.path.join(raw_dir, filename))
        im_mask = cv2.imread(os.path.join(mask_dir, train_id + ".png"), cv2.IMREAD_GRAYSCALE)

        # Apply mask on all
        im_raw[im_mask==0, ...] = 0
        
        # Generate blocks and save them
        for i, braw in enumerate(split_in_blocks(im_raw)):
            cv2.imwrite(os.path.join(block_dir, "{tid}_{bid}.jpg".format(tid=train_id, bid=i)), braw)

In [17]:
root_dir = "/home/ubuntu/sealion/data/"

In [ ]:
generate_training_blocks(root_dir)